In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import math
import matplotlib.pylab as plt
from scipy.stats import pearsonr
from scipy.spatial.distance import euclidean
from math import isnan
import time
import datetime

In [2]:
folderData = 'data\\'

First, testing with one particular example:

In [3]:
inter1 = pd.read_pickle(folderData + 'interactions1Data.pkl')
inter1.shape

(6867579, 5)

In [4]:
inter1.head()

,user_id,item_id,interaction_type,created_at,created_at_date
0,2082156,80,1,1484299172,2017-01-13 09:19:32
1,1934123,140,1,1486388563,2017-02-06 13:42:43
2,1320213,240,1,1479409825,2016-11-17 19:10:25
3,297303,310,1,1484817366,2017-01-19 09:16:06
4,1635596,310,1,1486370081,2017-02-06 08:34:41


In [5]:
grinter1 = inter1.groupby(['user_id','item_id']).size().reset_index()
grinter1.columns = ['user_id','item_id','number_int']
grinter1.shape

(5091857, 3)

In [6]:
grinter1.head()

,user_id,item_id,number_int
0,9,115748,1
1,9,536384,1
2,9,633504,1
3,9,720151,1
4,9,811877,1


In [7]:
grinter1.number_int.max()

116

It seems big.. Let's see a little bit more:

In [8]:
grinter1[grinter1.number_int > 50]

,user_id,item_id,number_int
1079067,497711,932451,52
1135904,524047,750429,67
1201330,553541,422684,60
1679599,774638,532405,83
1685120,777159,914031,58
1747634,806282,387909,116
1747637,806282,1423087,80
2108300,973017,1334342,53
2239260,1033686,760260,95
2991640,1381993,1570215,51


In [9]:
inter1[(inter1.user_id == 806282) & (inter1.item_id == 387909)].sort_values(by = 'created_at_date')

,user_id,item_id,interaction_type,created_at,created_at_date
9679877,806282,387909,1,1483635007,2017-01-05 16:50:07
9679900,806282,387909,1,1483642212,2017-01-05 18:50:12
9679943,806282,387909,1,1483645658,2017-01-05 19:47:38
9679947,806282,387909,1,1483646861,2017-01-05 20:07:41
9679908,806282,387909,1,1483648198,2017-01-05 20:29:58
9679942,806282,387909,1,1483648738,2017-01-05 20:38:58
9679891,806282,387909,1,1483650828,2017-01-05 21:13:48
9679876,806282,387909,1,1483720501,2017-01-06 16:35:01
9679915,806282,387909,1,1483720600,2017-01-06 16:36:40
9679940,806282,387909,1,1483720695,2017-01-06 16:38:15


Nevertheless, it seems credible: a given user can have clicked an item several times on the day...

## Better watching everyday

In [3]:
t0 = time.time()
inter_day = pd.DataFrame()
i = 0
for chunk in pd.read_csv(folderData + 'interactions.csv', sep='\t', iterator=True, chunksize=10000000):
    chunk.columns = [['user_id','item_id','interaction_type','created_at']]
    chunk['created_at_date'] = pd.to_datetime(arg=chunk.created_at,errors='coerce',unit='s')
    aux = (chunk['created_at_date']>='2016-11-06 00:00:00') & (chunk['created_at_date']<='2016-11-06 23:59:59')
    i += 1
    if i == 1:
        inter_day = chunk.loc[aux]
    else:
        inter_day = inter_day.append(chunk.loc[aux])
    print(str(i)+"...")
    
t1 = time.time()

print "Invested time: " + str((t1-t0)) + " s - "

1...
2...
3...
4...
5...
6...
7...
8...
9...
10...
11...
12...
13...
14...
15...
16...
17...
18...
19...
20...
21...
22...
23...
24...
25...
26...
27...
28...
29...
30...
31...
32...
33...
Invested time: 1951.82399988 s - 


In [6]:
inter_day.to_pickle(folderData + 'interactions_day\\interactions-2016-11-06.pkl')

And we do it for every day. Two days loading files!

Let's see the first day again.

In [10]:
folderint = folderData + '\\interactions_day\\'

In [11]:
inter_day0 = pd.read_pickle(folderint + 'interactions-2016-11-06.pkl')

In [12]:
grinter_day0 = inter_day0.groupby(['user_id','item_id']).size().reset_index()
grinter_day0.columns = ['user_id','item_id','number_int']
grinter_day0.shape

(26596, 3)

In [13]:
grinter_day0.head()

,user_id,item_id,number_int
0,179,1730422,1
1,525,1730422,1
2,530,1730422,1
3,691,1730422,1
4,893,1730422,1


In [14]:
grinter_day0.number_int.max()

8

Ok, relax about the first day!

## Now we can check the possible outliers

In [21]:
def open_day(date):
    return pd.read_pickle(folderint + 'interactions-' + date + '.pkl')

def reduce_day(date):
    aux = open_day(date)
    aux = aux.groupby(['user_id','item_id','interaction_type','created_at','created_at_date']).size().reset_index()
    aux.columns = ['user_id','item_id','interaction_type','created_at','created_at_date','number']
    aux = aux.drop(['number'],axis=1)
    return aux

def check_all(date):
    aux = open_day(date)
    aux = aux.groupby(['user_id','item_id']).size().reset_index()
    aux.columns = ['user_id','item_id','number_int']
    return aux#.number_int.max()

def check_out(date):
    aux = check_all(date)
    return aux.number_int.max()

def check_int(date):
    aux = open_day(date)
    aux = aux.groupby(['user_id','item_id','interaction_type']).size().reset_index()
    aux.columns = ['user_id','item_id','interaction_type','number_repeat']
    return aux#.number_repeat.max()

Ok let's go!

In [16]:
len(open_day('2016-11-06')), len(reduce_day('2016-11-06'))

(26750, 26750)

In [17]:
check_int('2016-11-06').number_repeat.max()

7

In [18]:
check_int('2016-11-06')[check_int('2016-11-06').number_repeat >= 7]

,user_id,item_id,interaction_type,number_repeat
17815,1545665,30425,1,7


In [19]:
open_day('2016-11-06')[(open_day('2016-11-06').user_id == 1545665) & (open_day('2016-11-06').item_id == 30425)]

,user_id,item_id,interaction_type,created_at,created_at_date
4237372,1545665,30425,1,1478475625,2016-11-06 23:40:25
4237375,1545665,30425,1,1478475861,2016-11-06 23:44:21
4237376,1545665,30425,1,1478475738,2016-11-06 23:42:18
4237378,1545665,30425,1,1478475849,2016-11-06 23:44:09
4237379,1545665,30425,1,1478475587,2016-11-06 23:39:47
4237380,1545665,30425,1,1478475544,2016-11-06 23:39:04
4237381,1545665,30425,2,1478475714,2016-11-06 23:41:54
4237382,1545665,30425,1,1478475975,2016-11-06 23:46:15


And let's compute the max number of the grouped interactions per day (yes, all the days!).

In [24]:
check_out('2016-11-07')

18

In [25]:
check_out('2016-11-08')

15

In [28]:
check_out('2016-11-09')

44

In [29]:
check_out('2016-11-10')

16

In [30]:
check_out('2016-11-11')

20

In [31]:
check_out('2016-11-12')

21

In [32]:
check_out('2016-11-13')

25

In [33]:
check_out('2016-11-14')

19

In [34]:
check_out('2016-11-15')

15

In [35]:
check_out('2016-11-16')

18

In [36]:
check_out('2016-11-17')

22

In [37]:
check_out('2016-11-18')

31

In [38]:
check_out('2016-11-19')

12

In [39]:
check_out('2016-11-20')

18

Not necessary checking the empty files: first, because they are empty; next, because I already did it by myself... The unique value is nan.

In [40]:
check_out('2016-12-12')

243

In [41]:
check_out('2016-12-13')

351

In [43]:
check_out('2016-12-14')

307

In [44]:
check_out('2016-12-15')

15

In [45]:
check_out('2016-12-16')

97

In [46]:
check_out('2016-12-17')

nan

In [47]:
check_out('2016-12-18')

2

This one doesn't seem empty although its memory is 1k:

In [48]:
pd.read_pickle(folderint + 'interactions-2016-12-18.pkl')

,user_id,item_id,interaction_type,created_at,created_at_date
9762587,2105774,663419,5,1482088401,2016-12-18 19:13:21
9762590,2105774,663419,5,1482088401,2016-12-18 19:13:21


There are repeated (same) interactions. This gonna give a first way to clean the interactions data.

In [49]:
check_out('2016-12-19')

133

In [50]:
check_out('2016-12-20')

307

In [51]:
check_out('2016-12-21')

273

In [52]:
check_out('2016-12-22')

84

In [53]:
check_out('2016-12-23')

442

In [54]:
check_out('2016-12-24')

nan

In [55]:
check_out('2016-12-25')

7

In [56]:
check_out('2016-12-26')

334

In [57]:
check_out('2016-12-27')

135

In [58]:
check_out('2016-12-28')

106

In [59]:
check_out('2016-12-29')

207

In [60]:
check_out('2016-12-30')

233

In [61]:
check_out('2016-12-31')

156

In [62]:
check_out('2017-01-01')

64

In [63]:
check_out('2017-01-02')

3164

Big! And it's just the beginning...

In [64]:
check_out('2017-01-03')

2708

In [65]:
check_out('2017-01-04')

2504

In [66]:
check_out('2017-01-05')

1745

In [67]:
check_out('2017-01-06')

3151

In [68]:
check_out('2017-01-07')

529

In [69]:
check_out('2017-01-08')

684

In [70]:
check_out('2017-01-09')

2943

In [71]:
check_out('2017-01-10')

1631

In [72]:
check_out('2017-01-11')

2367

In [73]:
check_out('2017-01-12')

3283

In [74]:
check_out('2017-01-13')

3173

In [75]:
check_out('2017-01-14')

606

In [76]:
check_out('2017-01-15')

500

In [77]:
check_out('2017-01-16')

2772

In [78]:
check_out('2017-01-17')

5209

Uf! This one is (I think) the biggest one! Let's watch it deeper...

In [79]:
machin = open_day('2017-01-17')

In [80]:
machin.shape

(17358843, 5)

In [81]:
check_all('2017-01-17').head()

,user_id,item_id,number_int
0,7,100419,5
1,7,309822,10
2,7,364664,8
3,7,636644,5
4,7,655513,10


In [82]:
check_int('2017-01-17').number_repeat.max()

4999

Let's see which is:

In [83]:
check_int('2017-01-17')[check_int('2017-01-17').number_repeat == 4999]

,user_id,item_id,interaction_type,number_repeat
2634588,1881755,1928479,5,4999


In [84]:
len(machin[(machin.user_id == 1881755) & (machin.item_id == 1928479) & (machin.interaction_type == 5)])

4999

There are 4999 times the same interaction: same user, same item, same interaction type and same date!

In [90]:
machin[(machin.user_id == 1881755) & (machin.item_id == 1928479) & (machin.interaction_type == 5)]

,user_id,item_id,interaction_type,created_at,created_at_date
208077,1881755,1928479,5,1484650417,2017-01-17 10:53:37
208078,1881755,1928479,5,1484650417,2017-01-17 10:53:37
208079,1881755,1928479,5,1484650417,2017-01-17 10:53:37
208082,1881755,1928479,5,1484650417,2017-01-17 10:53:37
208083,1881755,1928479,5,1484650417,2017-01-17 10:53:37
208084,1881755,1928479,5,1484650417,2017-01-17 10:53:37
208085,1881755,1928479,5,1484650417,2017-01-17 10:53:37
208086,1881755,1928479,5,1484650417,2017-01-17 10:53:37
208087,1881755,1928479,5,1484650417,2017-01-17 10:53:37
208088,1881755,1928479,5,1484650417,2017-01-17 10:53:37


Let's apply the reduction function and let's compare their respective lengths:

In [91]:
truc = reduce_day('2017-01-17')

In [92]:
print "Respective lengths before and after reduction: " + str(len(machin)) + ", " + str(len(truc))
print "Difference: " + str(len(machin) - len(truc))

Respective lengths before and after reduction: 17358843, 11932817
Difference: 5426026


Now let's finish the remaining ones.

In [93]:
check_out('2017-01-18')

1305

In [94]:
check_out('2017-01-19')

2602

In [95]:
check_out('2017-01-20')

2769

In [96]:
check_out('2017-01-21')

539

In [97]:
check_out('2017-01-22')

114

In [98]:
check_out('2017-01-23')

735

In [99]:
check_out('2017-01-24')

1313

In [100]:
check_out('2017-01-25')

1366

In [101]:
check_out('2017-01-26')

2706

In [102]:
check_out('2017-01-27')

1958

In [103]:
check_out('2017-01-28')

1096

In [104]:
check_out('2017-01-29')

1732

In [105]:
check_out('2017-01-30')

61

In [106]:
check_out('2017-01-31')

467

In [107]:
check_out('2017-02-01')

37

In [108]:
check_out('2017-02-02')

104

In [109]:
check_out('2017-02-03')

95

In [110]:
check_out('2017-02-04')

44

In [111]:
check_out('2017-02-05')

48

In [112]:
check_out('2017-02-06')

34

In [113]:
check_out('2017-02-07')

2

With the above example of abusive repetitions, we decide to reduce all the daily interactions files with the following way:

In [114]:
# reduce_day('YYYY-MM-DD').to_pickle(folderData + 'reduced_interactions_day\\reduced_interactions-YYYY-MM-DD.pkl')

We could also reduce the whole file interactions by the following way, but the laptop gets blocked. Too much...

t0 = time.time()
interactions_reduced = pd.DataFrame()
i = 0
for chunk in pd.read_csv(folderData + 'interactions.csv', sep='\t', iterator=True, chunksize=10000000):
    chunk.columns = [['user_id','item_id','interaction_type','created_at']]
    chunk['created_at_date'] = pd.to_datetime(arg=chunk.created_at,errors='coerce',unit='s')
    chunk = chunk.groupby(['user_id','item_id','interaction_type','created_at','created_at_date']).size().reset_index()
    chunk.columns = ['user_id','item_id','interaction_type','created_at','created_at_date','number']
    chunk = chunk.drop(['number'],axis=1)
    i += 1
    if i == 1:
        interactions_reduced = chunk
        print(str(i)+"...")
    else:
        interactions_reduced = interactions_reduced.append(chunk)
        print(str(i)+"...")
    
t1 = time.time()

print "Invested time: " + str((t1-t0)) + " s - "

Then let's reduce the partial files:

In [115]:
folderPartial = folderData + "reduced_partial_interactions\\"

In [116]:
folderData

'C:\\Users\\axiri\\Desktop\\postgrau\\projecte\\'

In [126]:
def reduce_data(dataset):
    dataset['created_at_date'] = pd.to_datetime(arg=dataset.created_at,errors='coerce',unit='s')
    aux = dataset.groupby(['user_id','item_id','interaction_type','created_at','created_at_date']).size().reset_index()
    aux.columns = ['user_id','item_id','interaction_type','created_at','created_at_date','number']
    aux = aux.drop(['number'],axis=1)
    return aux

In [117]:
interactions_bookmarked = pd.read_csv(folderData + "interactions_bookmarked.csv")

In [130]:
red_inter_book = reduce_data(interactions_bookmarked)

In [131]:
len(interactions_bookmarked), len(red_inter_book)

(281672, 281672)

Same lengths.

In [133]:
interactions_deleted = pd.read_csv(folderData + 'interactions_deleted.csv')

In [135]:
red_inter_del = reduce_data(interactions_deleted)

In [137]:
len(interactions_deleted), len(red_inter_del)

(906836, 906836)

Same lengths.

In [138]:
interactions_replied = pd.read_csv(folderData + 'interactions_replied.csv')

In [140]:
red_inter_rep = reduce_data(interactions_replied)

In [142]:
len(interactions_replied), len(red_inter_rep)

(117843, 117843)

Same lengths.

In [143]:
interactions_recruiter = pd.read_csv(folderData + 'interactions_recruiter.csv')

In [145]:
red_inter_recr = reduce_data(interactions_recruiter)

AttributeError: 'DataFrame' object has no attribute 'created_at'

Ok, let's extract it again!

In [148]:
t0 = time.time()
interactions_recruiter = pd.DataFrame()
i = 0
for chunk in pd.read_csv(folderData + 'interactions.csv', sep='\t', iterator=True, chunksize=10000000):
    chunk.columns = [['user_id','item_id','interaction_type','created_at']]
    chunk['created_at_date'] = pd.to_datetime(arg=chunk.created_at,errors='coerce',unit='s')
    aux = chunk.interaction_type == 5
    i += 1
    if i == 1:
        interactions_recruiter = chunk.loc[aux]
        print(str(i)+"...")
    else:
        interactions_recruiter = interactions_recruiter.append(chunk.loc[aux])
        print(str(i)+"...")
    
t1 = time.time()

print "Invested time: " + str((t1-t0)) + " s - "

1...
2...
3...
4...
5...
6...
7...
8...
9...
10...
11...
12...
13...
14...
15...
16...
17...
18...
19...
20...
21...
22...
23...
24...
25...
26...
27...
28...
29...
30...
31...
32...
33...
Invested time: 1673.51099992 s - 


In [150]:
interactions_recruiter.shape

(100971, 5)

In [151]:
red_inter_recr = reduce_data(interactions_recruiter)

In [153]:
red_inter_recr.head()

,user_id,item_id,interaction_type,created_at,created_at_date
0,2928,219645,5,1482404881,2016-12-22 11:08:01
1,2928,621052,5,1482404881,2016-12-22 11:08:01
2,7925,1945852,5,1484823078,2017-01-19 10:51:18
3,9332,1072249,5,1485518678,2017-01-27 12:04:38
4,14865,689566,5,1485444051,2017-01-26 15:20:51


In [154]:
len(interactions_recruiter), len(red_inter_recr)

(100971, 2554)

In [155]:
other_inter_recr = pd.read_csv(folderData + 'interactions_recruiter.csv')

In [156]:
len(other_inter_recr)

100971

In [157]:
red_inter_recr.to_pickle(folderPartial + 'red_inter_recr.pkl')

Let's see about 1-interaction:

In [159]:
interactions1Data = pd.read_pickle(folderData + 'interactions1Data.pkl')

In [160]:
interactions1Data.interaction_type.unique()

array([1], dtype=int64)

In [161]:
red_inter1 = reduce_data(interactions1Data)

In [164]:
len(interactions1Data), len(red_inter1)

(6867579, 6867579)

Don't need to reduce...